In [48]:
import importlib
import Parallel_pair_correlation_HDF5
importlib.reload(Parallel_pair_correlation_HDF5)
from Parallel_pair_correlation_HDF5 import *

import matplotlib.pyplot as plt
import sys

sys.path.append('/home/hcleroy/PostDoc/aging_condensates/Simulation/Gillespie/Analysis/')
sys.path.append('/home/hugo/PostDoc/aging_condensates/Gillespie/Analysis/')
from ToolBox import *

sys.path.append('/home/hcleroy/PostDoc/aging_condensates/Simulation/Gillespie/Gillespie_backend')
import Gillespie_backend as Gil

sys.path.append('/home/hcleroy/Extra_Module_py')
sys.path.append('/home/hugo/Extra_Module_py')
import formated_matplotlib

Ec = lambda L,N : 3/2*np.log(L/N* np.pi/3)

In [49]:
# gillespie parameter
Nlinker = 20
ell_tot = 10**3
V = 2*((ell_tot/6)**1.5/Nlinker**0.5*3/4*np.pi)
kdiff = 100/(V) # for 3D
Energy = -15
Nprocess = 100
args = [[ell_tot,Energy,kdiff,np.random.randint(1000000),Nlinker,3] for _ in range(Nprocess)]
print(Ec(ell_tot,Nlinker))
# Simulation parameters
step_tot = 100000
compute_steps = 30

5.937210903914155


In [50]:
Parallel_correlation_function(args,step_tot,1000,"20_linker_Ell_10E3.hdf",30,np.sqrt(ell_tot))

Process Process-53:
Traceback (most recent call last):
  File "/home/hcleroy/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hcleroy/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/hcleroy/PostDoc/aging_condensates/Simulation/Evolution_of_pair_prob_distrib/Parallel_pair_correlation_HDF5.py", line 95, in handle_output
    hdf = pt.open_file(filename, mode='w') # open a hdf5 file
  File "/home/hcleroy/anaconda3/lib/python3.8/site-packages/tables/file.py", line 300, in open_file
    return File(filename, mode, title, root_uep, filters, **kwargs)
  File "/home/hcleroy/anaconda3/lib/python3.8/site-packages/tables/file.py", line 750, in __init__
    self._g_new(filename, mode, **params)
  File "tables/hdf5extension.pyx", line 486, in tables.hdf5extension.File._g_new
tables.exceptions.HDF5ExtError: HDF5 error back trace

  File "H5F.c", line 444, in H5Fcr

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/hcleroy/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_19722/2204262003.py", line 1, in <module>
    Parallel_correlation_function(args,step_tot,1000,"20_linker_Ell_10E3.hdf",30,np.sqrt(ell_tot))
  File "/home/hcleroy/PostDoc/aging_condensates/Simulation/Evolution_of_pair_prob_distrib/Parallel_pair_correlation_HDF5.py", line 149, in Parallel_correlation_function
    p.join()
  File "/home/hcleroy/anaconda3/lib/python3.8/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/home/hcleroy/anaconda3/lib/python3.8/multiprocessing/popen_fork.py", line 47, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/home/hcleroy/anaconda3/lib/python3.8/multiprocessing/popen_fork.py", line 27, in poll
    pid, sts = os.waitpid(self.pid, flag)
KeyboardInterrupt

During ha

In [41]:
def histogram_float(*args, **kwargs):
    counts, bin_edges = np.histogram(*args, **kwargs)
    return counts.astype(float), bin_edges

def unit_test(gillespie, check_steps = 100, step_tot=10**3, num_bins=30, max_distance=30):
    counts, bin_edges = histogram_float([], bins=num_bins, range=(0, max_distance))
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    bin_widths = bin_edges[1:] - bin_edges[:-1]
    shell_volumes = (4 / 3) * np.pi * ((bin_centers + bin_widths)**3 - bin_centers**3)
    for i in range(step_tot//check_steps):
        counts, bin_edges = histogram_float([], bins=num_bins, range=(0, max_distance))
        prev_hist = np.zeros(counts.shape,dtype=float)
        t_tot = 0.
        for t in range(check_steps):
            move, time = gillespie.evolve()
            t_tot +=  time
            counts += prev_hist * time
            dist = np.linalg.norm(gillespie.get_R()[1:]-gillespie.get_R()[:-1])
            prev_hist, bin_edges = histogram_float(dist, bins=num_bins, range=(0, max_distance))
        counts = counts / (t_tot * shell_volumes)
    
    return np.stack((counts, bin_centers), axis=-1)


In [39]:
gillespie = Gil.Gillespie(rho0 = 0,Nlinker=3) 

In [40]:
unit_test(gillespie)

array([[3.57524898e-02, 5.00000000e-01],
       [0.00000000e+00, 1.50000000e+00],
       [1.37101842e-03, 2.50000000e+00],
       [1.09769184e-03, 3.50000000e+00],
       [3.38849375e-04, 4.50000000e+00],
       [5.77348360e-05, 5.50000000e+00],
       [0.00000000e+00, 6.50000000e+00],
       [0.00000000e+00, 7.50000000e+00],
       [0.00000000e+00, 8.50000000e+00],
       [0.00000000e+00, 9.50000000e+00],
       [0.00000000e+00, 1.05000000e+01],
       [0.00000000e+00, 1.15000000e+01],
       [0.00000000e+00, 1.25000000e+01],
       [0.00000000e+00, 1.35000000e+01],
       [0.00000000e+00, 1.45000000e+01],
       [0.00000000e+00, 1.55000000e+01],
       [0.00000000e+00, 1.65000000e+01],
       [0.00000000e+00, 1.75000000e+01],
       [0.00000000e+00, 1.85000000e+01],
       [0.00000000e+00, 1.95000000e+01],
       [0.00000000e+00, 2.05000000e+01],
       [0.00000000e+00, 2.15000000e+01],
       [0.00000000e+00, 2.25000000e+01],
       [0.00000000e+00, 2.35000000e+01],
       [0.000000

In [23]:
np.histogram([])[0].dtype

dtype('int64')